# Pipeline to prepare ligand files

In [1]:
prot_name = 'cdk2'

In [2]:
DIR_LIGS = F'../../ARCHIVOS/CRISTALES/LIGS_{prot_name.upper()}'
dir_inputs = F'{DIR_LIGS}/IN_POCKET_NON_PREP/'
dir_outputs_sdf = F'{DIR_LIGS}/POCKET_LIGS_PREP_SDF/'
dir_outputs_mol2 = F'{DIR_LIGS}/POCKET_LIGS_PREP_MOL2/'

### Primer paso: creamos los directorios de salida

In [3]:
import os
for i in [dir_outputs_mol2, dir_outputs_sdf]:
    print(i)
    if not os.path.exists(i):
        os.makedirs(i)
        print('Creado...')

../../ARCHIVOS/CRISTALES/LIGS_CDK2/POCKET_LIGS_PREP_MOL2/
../../ARCHIVOS/CRISTALES/LIGS_CDK2/POCKET_LIGS_PREP_SDF/


### Segundo paso:

### Formato de las moléculas

<div style='background-color: #F5A566; padding: 5px 12px;'>Antes de cargar las moléculas, debemos convertirlas a un formato que sea correctamente leído con rdkit.</div>

<br>
<div class='bg-info' style='padding: 10px;'>
Por lo que se ha analizado hasta ahora, <span style='font-weight: bold;'> desde un archivo mol2 no funcionará</span>  si la molécula tiene ciclos que no siguen el estándar de corina ('C.ar' o 'N.ar'), u oxígenos unidos a fósforo debido a que rdkit no reconoce los Oxígenos O.c2 en archivos mol2 generados por babel o Chimera (formatos distintos al estándar 'corina' que sigue rdkit).<br>El problema no parece poder resolverse sustituyendo, por ejemplo, 'C.ar' por 'C.2', pues manualmente hay que elegir qué átomos deben tener un enlace <i>sp2</i> (C=C) o sp3 (C-C) dentro del ciclo.<a src='https://sourceforge.net/p/rdkit/mailman/message/28665746/'>Fuente</a>.
</div>


**Para ello en necesario considerar lo siguiente:**

Para usar rdkit, los archivos pueden ser leídos a partir de dos fuentes:
1. **`sdf` desde un pdb:** Desde un archivo pdb sin Hs, usando: `obabel -ipdb file.pdb -osdf file.sdf -p 7.4`
2. **`sdf` desde uel mol2:** Desde el archivo mol2 generado con `Chimera` que ya ha sido protonado con este mismo programa, usando: `obabel -imol2 file.mol2 -osdf file.sdf`. Es importante no usar el parámetro `-p`.

### Generamos los archivos .mol2 de las moléculas empleando UCSF Chimera

En algunas ocasiones ha resultado más conveniente generar las moléculas con hidrógenos utilizando UCSF Chimera (al menos cuando se espera que sea a pH neutro). Para ello usar el siguiente script llamado `chim_addh.py``:

```python
from chimera import runCommand as rc

from os import chdir, listdir
chdir("path/to/the/pdb_molecules/dir") 

for m in listdir('.'):
	rc('open ' + m)
	rc("addh")
	rc('write format mol2 0 ' + m[:-4] + '.mol2')
	rc('close all')
```

Y se ejecuta con el siguiente comando desde la terminal:

`chimera --nogui chim_addh.py`

Nota: chimera debe estar en el path, de otro modo hay que ejecutar el script añadiendo la dirección absoluta hacia el ejecutable de chimera.

### Creamos el script en el directorio actual:
La siguiente celda crea el script mencionado arriba.

In [4]:
with open('chim_addh.py', 'w') as f:
    f.write(F'''
from chimera import runCommand as rc

from os import chdir, listdir
chdir("{dir_inputs}") 

for m in listdir('.'):
    rc('open ' + m)
    rc("addh")
    rc('write format mol2 0 ' + m[:-4] + '.mol2')
    rc('close all')    
    ''')

### Ejecutamos el script con UCSF Chimera

In [ ]:
%%bash -s $dir_inputs $dir_outputs_mol2
# Ruta al aejecutable de chimera
chimera='/home/joel/.local/UCSF-Chimera64-1.14rc/bin/chimera'

$chimera --nogui chim_addh.py
wait
## Ahora movemos los archivos mol2 a la carpeta correspondiente
mv $1/*.mol2 $2

Como resultado tendríamos las moléculas mol2 en su carpeta correspondiente. el siguiente paso es usar babel, para generar los archivos *sdf*.

### Usamos OpenBabel para generar los archivos sdf (para usar con rdkit)


Los archivos pueden ser leídos a partir de dos fuentes:
1. **`sdf` desde un pdb:** Desde un archivo pdb sin Hs, usando: `obabel -ipdb file.pdb -osdf file.sdf -p 7.4`
2. **`sdf` desde uel mol2:** Desde el archivo mol2 generado con `Chimera` que ya ha sido protonado con este mismo programa, usando: `obabel -imol2 file.mol2 -osdf file.sdf`. Es importante no usar el parámetro `-p`

```bash
# Usamos obabel 
cd "../ARCHIVOS/CRISTALES/LIGS_CDK2/IN_POCKET_NON_PREP"
for mol in `ls *pdb`
do 
    name=${mol%.*}
    # Directamente desde el archivo pdb sin Hs
    obabel -ipdb $mol -osdf -O ${name}_from_pdb.sdf -p 7.4 # Añade Hs a ph fisiológico
done

mv *sdf ../LIGS_POSES_SDF/

cd "../LIGS_PREP_IN_POCKET"
for mol in `ls *mol2`
do 
    name=${mol%.*}
    # directamente desde el archivo mol2 generado con chimera y sin el parámetro -p
    obabel -imol2 $mol -osdf -O ${name}_from_mol2.sdf
done

mv *sdf ../LIGS_POSES_SDF/
```

### Ejecutamos obabel

In [9]:
mol2_dir = dir_outputs_mol2.split('/')[-2]
sdf_dir = dir_outputs_sdf.split('/')[-2]

In [ ]:
%%bash -s $dir_inputs $sdf_dir $mol2_dir
cd $1
for mol in `ls *pdb`
do 
    name=${mol%.*}
    # Directamente desde el archivo pdb sin Hs
    obabel -ipdb $mol -osdf -O ${name}_from_pdb.sdf -p 7.4 # Añade Hs a ph fisiológico
done
# Mueve los archivos al directorio de sdfs
mv *.sdf ../$2

# Nos movemos al directorio con archivos mol2
cd ../$3
for mol in `ls *mol2`
do 
    name=${mol%.*}
    # directamente desde el archivo mol2 generado con chimera y sin el parámetro -p
    obabel -imol2 $mol -osdf -O ${name}_from_mol2.sdf
done
# Mueve los archivos al directorio de sdfs
mv *.sdf ../$2

Cuando la molécula provenga desde un archivo *pdbqt* o *pdb* de docking, el proceso para obtener los archivos superiores es primero remover los hidrógenos de la molécula y luego añadirlos con `Chimera` o `babel` (en teoría, antes del docking se habría hecho el mismo proceso para tener el estado de protonación correcto). Finalemente se guardará la molécula en un archivo **`mol2`** que correspoonde al solicitado en el punto #1 de la lista anterior.

## ¡Terminado!
El análisis continúa en el notbooks de cálculo de descriptores con *rdkit*.